In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2, r_regression
from torch.utils.data import DataLoader, Dataset, TensorDataset, WeightedRandomSampler
from sklearn.metrics import RocCurveDisplay, f1_score, PrecisionRecallDisplay
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [ ]:
# Dataset can be found here: https://www.kaggle.com/datasets/hassan06/nslkdd

feature=["duration","protocol_type","service","flag","src_bytes","dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins","logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations","num_shells","num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count","srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate","diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count","dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate","target", "difficulty"]
df1 = pd.read_csv('<filepath>/datasets/KDDTest+.txt', names=feature)
df2 = pd.read_csv('<filepath>/datasets/KDDTrain+.txt', names=feature)

df = pd.concat([df1, df2])
df.shape

In [ ]:
# Drop completely homogenous columns

df.drop([col for col in df.select_dtypes(include=[np.number]) if df[col].max()-df[col].min() == 0], axis=1, inplace=True)

In [ ]:
# Drop target classes with less than 100 samples

targets_to_drop = [val for val in df['target'].unique() if len(df[df['target'] == val]) < 100]
df.drop(df.index[df['target'].isin(targets_to_drop)], inplace=True)

df[['target']].value_counts()

In [ ]:
# Normalization
df_nums = df.select_dtypes(include=[np.number])
df_nums = (df_nums-df_nums.min())/(df_nums.max()-df_nums.min())

for col in df_nums.columns:
    df[col] = df_nums[col]

df.dropna(axis=0, inplace=True)
for c in df.columns:
    print(c + " :", df[c].isna().sum())

In [ ]:
df[['target']].value_counts()

In [ ]:
# 1 denotes OOD, 0 denotes ID

ood_cats = ["httptunnel", "snmpgetattack", "apache2"]
def mark_ood(row):
    if row['target'] in ood_cats: row['ood'] = 1
    else: row['ood'] = 0
    return row

df = df.apply(mark_ood, axis=1)

In [ ]:
# Split dataset into train, test, OOD

df_temp_X = df.drop(['target'], axis=1)
df_temp_y = df[['target', 'ood']]
df_temp_X = pd.get_dummies(df_temp_X)

X = df_temp_X[df_temp_X['ood'] == 0]
X.drop(['ood'], inplace=True, axis=1)

y = df_temp_y[df_temp_y['ood'] == 0]
y.drop(['ood'], inplace=True, axis=1)
y = pd.get_dummies(y)

X_ood = df_temp_X[df_temp_X['ood'] == 1]
X_ood.drop(['ood'], inplace=True, axis=1)
ood_len = len(X_ood)

X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.2)

X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.FloatTensor(y_train)
y_test = torch.FloatTensor(y_test)
X_ood = torch.FloatTensor(np.concatenate((X_test, X_ood.values), axis=0))
y_ood = torch.FloatTensor([1 for j in range(len(X_test))] + [0 for i in range(ood_len)]) # 1 == ID, 0 == OOD

weightlist = [1 / y[col].sum() for col in y.columns]
samp = WeightedRandomSampler(weights=[weightlist[cat.argmax()] for cat in y_train], num_samples=len(y_train), replacement=True)

train = TensorDataset(X_train, y_train)
train_loader = DataLoader(train, batch_size=512, sampler=samp)

test = TensorDataset(X_test, y_test)
test_loader = DataLoader(test, batch_size=512)

ood = TensorDataset(X_ood, y_ood)
ood_loader = DataLoader(ood, batch_size=512)

In [ ]:
for x, y in train_loader:
    print(x.shape)
    print(y.shape)
    break

In [ ]:
# Verify the distribution of classes are uniform

from collections import defaultdict
store = defaultdict(int)
for x, y in train_loader:
    for val in y:
        store[int(val.argmax(0))] += 1
    break
for key in store:
    print(key, store[key])

In [ ]:
# Neural network

class ANN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(in_features=122, out_features=128)
        self.norm = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(in_features=128, out_features=128)
        self.norm2 = nn.BatchNorm1d(128)
        self.output = nn.Linear(in_features=128, out_features=18)
        #self.output2 = nn.Softmax(dim=1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.norm(x)
        x = F.relu(self.fc2(x))
        x = self.norm2(x)
        #x = self.norm4(x)
        x = self.output(x)
        #x = self.output2(x)
        return x

model = ANN()
model

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
# Training phase

epochs = 30
lossarr = []
size = len(test_loader.dataset)
num_batches = len(test_loader)
testloss = []
for i in range(epochs):
    model.train()
    for batch, (x, y) in enumerate(train_loader):
        y_hat = model.forward(x)
        loss = criterion(y_hat, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    lossarr.append(loss.item())
    
    model.eval()
    test_loss, correct = 0, 0
    preds = []
    with torch.no_grad():
        for X, y in test_loader:
            pred = model(X)
            preds.extend(pred)
            test_loss += criterion(pred, y).item()
            correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
    testloss.append(test_loss/num_batches)

plt.plot(list(range(epochs)), lossarr, label="training loss")
plt.plot(list(range(epochs)), testloss, label="test loss")
plt.legend()
plt.show()

In [ ]:
# Testing phase

size = len(test_loader.dataset)
num_batches = len(test_loader)
model.eval()
test_loss, correct = 0, 0
preds = []
with torch.no_grad():
    for X, y in test_loader:
        pred = model(X)
        preds.extend(pred)
        test_loss += criterion(pred, y).item()
        correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
test_loss /= num_batches
correct /= size
print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
from collections import defaultdict
store = defaultdict(int)
for x, y in test_loader:
    for val in y:
        store[int(val.argmax(0))] += 1
for key in store:
    print(key, store[key])

In [ ]:
# Plot the ROC curve

plt.figure(figsize=(11,11))
ax = plt.gca()
sm = nn.Softmax(dim=0)
for atk in range(18):
    labels = [int(val[atk]) for X, y in test_loader for val in y]
    probs = [float(sm(tensor)[atk]) for tensor in preds]
    RocCurveDisplay.from_predictions(labels, probs, name=f"Class: {atk}",ax=ax)
    print(f"F1 score for Class {atk} :", f1_score(labels, [int(int(tensor.argmax(0)) == atk) for tensor in preds]))
plt.show()

In [ ]:
# ROC for OOD evaluation. Positive class is ID

model.eval()
preds = []
sm = nn.Softmax(dim=0)
labels = []
with torch.no_grad():
    for X, y in ood_loader:
        pred = model(X)
        preds.extend(pred)
        labels.extend(y)

plt.figure(2, figsize=(11,11))
ax = plt.gca()
msp = [float(max(sm(tensor))) for tensor in preds]
RocCurveDisplay.from_predictions(labels, msp, ax=ax)
plt.show(2)

In [ ]:
# ROC for OOD evaluation. Positive class is OOD

model.eval()
preds = []
sm = nn.Softmax(dim=0)
labels = []
with torch.no_grad():
    for X, y in ood_loader:
        pred = model(X)
        preds.extend(pred)
        labels.extend(y)

plt.figure(3, figsize=(11,11))
ax = plt.gca()
msp = [-float(max(sm(tensor))) for tensor in preds]
RocCurveDisplay.from_predictions([0 if i == 1 else 1 for i in labels], msp, ax=ax)
plt.show(3)

In [ ]:
# PR for OOD evaluation. Positive class is ID

model.eval()
preds = []
sm = nn.Softmax(dim=0)
labels = []
with torch.no_grad():
    for X, y in ood_loader:
        pred = model(X)
        preds.extend(pred)
        labels.extend(y)

plt.figure(4, figsize=(11,11))
ax = plt.gca()
msp = [float(max(sm(tensor))) for tensor in preds]
PrecisionRecallDisplay.from_predictions(labels, msp, ax=ax)
plt.show(4)

In [ ]:
# PR for OOD evaluation. Positive class is OOD

model.eval()
preds = []
sm = nn.Softmax(dim=0)
labels = []
with torch.no_grad():
    for X, y in ood_loader:
        pred = model(X)
        preds.extend(pred)
        labels.extend(y)

plt.figure(3, figsize=(11,11))
ax = plt.gca()
msp = [-float(max(sm(tensor))) for tensor in preds]
PrecisionRecallDisplay.from_predictions([0 if i == 1 else 1 for i in labels], msp, ax=ax)
plt.show(3)